# Areas of Interest

ArcGIS Pro has been used to create rectangular polygon features around areas of interest, which will be used to reduce the computation of satellite data and other sources.

This notebook simply converts these features from vertices in a CSV file to a dictionary of values that can be used for the `geoRegion` parameter of the Subset node of a SNAP graph. Edges are corrected to be exactly vertical or exactly horizontal to improve efficiency.

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('AOIs.csv', header=None)

In [4]:
df.head()

,0,1,2,3
0,-5.221344,49.990184,30UUA,2
1,-5.224443,50.286487,30UUA,2
2,-4.885354,50.290033,30UUA,2
3,-4.882255,49.993731,30UUA,2
4,-5.221344,49.990184,30UUA,2


Each polygon has a unique value in column 3.

In [5]:
df[3].unique()

array([ 2,  3,  4,  6,  9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
       23, 24, 25, 26, 27, 28])

In [6]:
df[df[3]==2]

,0,1,2,3
0,-5.221344,49.990184,30UUA,2
1,-5.224443,50.286487,30UUA,2
2,-4.885354,50.290033,30UUA,2
3,-4.882255,49.993731,30UUA,2
4,-5.221344,49.990184,30UUA,2


We need the extremes of longitude and latitude for each polygon.

In [7]:
[
    df[df[3]==2][0].min(),
    df[df[3]==2][1].min(),
    df[df[3]==2][0].max(),
    df[df[3]==2][1].max()
]

[-5.22444257, 49.9901843, -4.88225479, 50.29003327]

In [8]:
bbox = [
    df[df[3]==2][0].min(),
    df[df[3]==2][1].min(),
    df[df[3]==2][0].max(),
    df[df[3]==2][1].max()
]

Then we get the vertices of our `geoRegion`.

In [14]:
f"POLYGON (({bbox[0]} {bbox[1]}, {bbox[2]} {bbox[1]}, {bbox[2]} {bbox[3]}, {bbox[0]} {bbox[3]}, {bbox[0]} {bbox[1]}))"

'POLYGON ((-5.22444257 49.9901843, -4.88225479 49.9901843, -4.88225479 50.29003327, -5.22444257 50.29003327, -5.22444257 49.9901843))'

In [15]:
def poly_to_geoRegion(p):
    bbox = [
        df[df[3]==p][0].min(),
        df[df[3]==p][1].min(),
        df[df[3]==p][0].max(),
        df[df[3]==p][1].max()
    ]
    return f"POLYGON (({bbox[0]} {bbox[1]}, {bbox[2]} {bbox[1]}, {bbox[2]} {bbox[3]}, {bbox[0]} {bbox[3]}, {bbox[0]} {bbox[1]}))"

We also need the tile_Id from column 2.

In [16]:
df[df[3]==2][2][0]

'30UUA'

In [21]:
poly_dict = {str(p): {'geoRegion': poly_to_geoRegion(p), 'tile_Id': df[df[3]==p][2].iloc[0]} for p in df[3].unique()}

In [22]:
poly_dict

{'2': {'geoRegion': 'POLYGON ((-5.22444257 49.9901843, -4.88225479 49.9901843, -4.88225479 50.29003327, -5.22444257 50.29003327, -5.22444257 49.9901843))',
  'tile_Id': '30UUA'},
 '3': {'geoRegion': 'POLYGON ((-4.81917128 50.19269224, -4.59558622 50.19269224, -4.59558622 50.40467681, -4.81917128 50.40467681, -4.81917128 50.19269224))',
  'tile_Id': '30UUA'},
 '4': {'geoRegion': 'POLYGON ((-4.32949749 50.26538565, -4.02885352 50.26538565, -4.02885352 50.5075424, -4.32949749 50.5075424, -4.32949749 50.26538565))',
  'tile_Id': '30UVA'},
 '6': {'geoRegion': 'POLYGON ((-4.02836874 50.14143345, -3.59050258 50.14143345, -3.59050258 50.34954708, -4.02836874 50.34954708, -4.02836874 50.14143345))',
  'tile_Id': '30UVA'},
 '9': {'geoRegion': 'POLYGON ((-3.70217003 50.32171356, -3.4070158 50.32171356, -3.4070158 50.54736093, -3.70217003 50.54736093, -3.70217003 50.32171356))',
  'tile_Id': '30UVA'},
 '11': {'geoRegion': 'POLYGON ((-4.42178593 50.96017888, -3.80799343 50.96017888, -3.80799343 51.

Save as JSON

In [23]:
import json

In [24]:
with open('AOIs.txt', 'w') as fp:
    json.dump(poly_dict, fp)